# Working with BIDS data

In [4]:
import matplotlib
import pathlib
import pooch

import mne
import mne_bids

matplotlib.use('Qt5Agg')

## Read the input data we wish to convert to BIDS

In [5]:
sample_data_dir = mne.datasets.sample.data_path()
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

Opening raw data file C:\Users\Ily\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
320 events found
Event IDs: [ 1  2  3  4  5 32]


## Write the raw data to BIDS!

We need to specify the power line frequency.

In [6]:
raw.info['line_freq'] = 60

Now, actually write the data.

In [9]:
out_path = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='02',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True)

Opening raw data file C:\Users\Ily\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.

Writing 'out_data\sample_BIDS\participants.tsv'...

participant_id	age	sex	hand
sub-01	n/a	n/a	n/a
sub-02	n/a	n/a	n/a

Writing 'out_data\sample_BIDS\participants.json'...

{
    "participant_id": {
        "Description": "Unique participant identifier"
    },
    "age": {
        "Description": "Age of the participant at time of testing",
        "Units": "years"
    },
    "sex": {
        "Description": "Biological sex of the participant",
        "Levels": {
            "F": "female",
            "M": "male"
        }
    },
    "hand": {
        "Description": "Handedness of the participant",
        "Levels": {
            "R": "right",
            "L": "left",
            "A": "ambidextr

BIDSPath(
root: out_data/sample_BIDS
datatype: meg
basename: sub-02_ses-01_task-audiovisual_run-01_meg.fif)

## Write MEGIN / Elekta / NeuroMag fine-calibration and crosstalk files

In [10]:
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path=bids_path)


Writing fine-calibration file to out_data/sample_BIDS/sub-02/ses-01/meg/sub-02_ses-01_acq-calibration_meg.dat
Writing crosstalk file to out_data/sample_BIDS/sub-02/ses-01/meg/sub-02_ses-01_acq-crosstalk_meg.fif


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Write the same raw data again, but this time act as if it were from a different subject – subject "02".</li>
    </ul>
</div>

## Print the created file & directory structure

In [11]:
mne_bids.print_dir_tree(out_path)

|sample_BIDS\
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01\
|------ ses-01\
|--------- sub-01_ses-01_scans.tsv
|--------- meg\
|------------ sub-01_ses-01_acq-calibration_meg.dat
|------------ sub-01_ses-01_acq-crosstalk_meg.fif
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.fif
|------------ sub-01_ses-01_task-audiovisual_run-01_meg.json
|--- sub-02\
|------ ses-01\
|--------- sub-02_ses-01_scans.tsv
|--------- meg\
|------------ sub-02_ses-01_acq-calibration_meg.dat
|------------ sub-02_ses-01_acq-crosstalk_meg.fif
|------------ sub-02_ses-01_coordsystem.json
|------------ sub-02_ses-01_task-audiovisual_run-01_channels.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_events.tsv
|------------ sub-02_ses-01_task-audiovisual_run-01_meg.fif
|-------

## Generate a data summary

In [12]:
print(mne_bids.make_report(out_path))

Summarizing participants.tsv out_data\sample_BIDS\participants.tsv...
Summarizing scans.tsv files [WindowsPath('out_data/sample_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv'), WindowsPath('out_data/sample_BIDS/sub-02/ses-01/sub-02_ses-01_scans.tsv')]...
The participant template found: sex were all unknown;
handedness were all unknown; ages all unknown
This dataset was created by [Unspecified] and conforms to BIDS version 1.6.0.
This report was generated with MNE-BIDS (https://doi.org/10.21105/joss.01896).
The dataset consists of 2 participants (sex were all unknown; handedness were
all unknown; ages all unknown) and 1 recording sessions: 01. Data was recorded
using a MEG system (Elekta manufacturer) sampled at 600.61 Hz with line noise at
60 Hz. There were 2 scans in total. Recording durations ranged from 277.71 to
277.71 seconds (mean = 277.71, std = 0.0), for a total of 555.43 seconds of data
recorded over all scans. For each dataset, there were on average 376.0 (std =
0.0) recording c

## Reading BIDS data

Let's try to read back the data we just stored according to the BIDS specification.

In [23]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.


<ipython-input-23-1122cbf357fe>:10: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


In [25]:
raw.plot()

<MNEBrowseFigure size 800x800 with 5 Axes>

Channels marked as bad: ['MEG 2443', 'EEG 053']


## Events are stored as annotations – but we convert between the two.

In [20]:
raw.annotations[0]

OrderedDict([('onset', 46.58058882905339),
             ('duration', 0.0),
             ('description', 'Auditory/Right'),
             ('orig_time',
              datetime.datetime(2002, 12, 3, 19, 1, 10, 720100, tzinfo=datetime.timezone.utc))])

In [21]:
events, event_id = mne.events_from_annotations(raw)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [22]:
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

<Figure size 640x480 with 1 Axes>

## Fine the MEG fine-calibration and crosstalk files

In [26]:
bids_path.meg_calibration_fpath

WindowsPath('out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-calibration_meg.dat')

In [27]:
bids_path.meg_crosstalk_fpath

WindowsPath('out_data/sample_BIDS/sub-01/ses-01/meg/sub-01_ses-01_acq-crosstalk_meg.fif')